In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("/Users/vazgenosipov/Desktop/Python/Python Data analys/UNZIP_FOR_NOTEBOOKS_FINAL/DATA/Advertising.csv")

df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


# Grid Search

Is a way of training and validating a model on every possible combination of multiple hyperparameter options

We can search through a variety of combinations of hyperparameters with a grid search. While many linear models are quite simple and even come with their own specialized versions that do a search for you, this method of a grid search will can be applied to *any* model from sklearn, and we will need to use it later on for more complex models, such as Support Vector Machines.

Scikit_learn includes a GridSearchCV class capable of testing a dictionary of multiple hyperparameter options through cross-validation 

This allows for both cross_validation and grid search to be performed in a generalized way for any model

In [6]:
# Create X and y
X = df.drop("sales", axis = 1)
y = df["sales"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)

# Scale the data 
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()

# dont forget about data leakege 
scale.fit(X_train)

# transform the data
X_train = scale.transform(X_train)
X_test = scale.transform(X_test)

In [7]:
# lets use Elastic net model that uses both L1(Lasso) and L2(Ridge) models 

from sklearn.linear_model import ElasticNet

### Grid Search

A search consists of:

* an estimator (regressor or classifier such as sklearn.svm.SVC());
* a parameter space;
* a method for searching or sampling candidates;
* a cross-validation scheme 
* a score function.

In [9]:
# first we need to create a dummy instance of the model without any parameters
base_elastic_net_model = ElasticNet()

In [14]:
# we now create hyperparameters to go through
# ElasicNet takes two L1 value (the use of Lasso in comparison to Ridge) and alpha value
# we use it in a from of a dictionary, 
# keys - parameters, values - hyperparameters list
# see the recomended list parameters to go through

param_grid = {"alpha": [0.1,0.25, 0.5, 1,2,5,10,50,100], 
              "l1_ratio": [0.1, 0.15,0.20,0.25,0.5,0.7,0.8,0.9,0.95,0.95,1]}

In [15]:
from sklearn.model_selection import GridSearchCV

In [28]:
# create an instance of grid

grid_model = GridSearchCV(estimator=base_elastic_net_model, param_grid=param_grid,
                         scoring="neg_mean_squared_error", cv=5,
                         verbose=1)

# verbose gives you the information, the
# the higher the number, the higher the granularity of the report

In [29]:
# now we fit the model to our data

grid_model.fit(X_train, y_train)

Fitting 5 folds for each of 99 candidates, totalling 495 fits


GridSearchCV(cv=5, estimator=ElasticNet(),
             param_grid={'alpha': [0.1, 0.25, 0.5, 1, 2, 5, 10, 50, 100],
                         'l1_ratio': [0.1, 0.15, 0.2, 0.25, 0.5, 0.7, 0.8, 0.9,
                                      0.95, 0.95, 1]},
             scoring='neg_mean_squared_error', verbose=1)

In [30]:
# now lets get the best estimator

grid_model.best_estimator_

ElasticNet(alpha=0.1, l1_ratio=1)

In [31]:
grid_model.best_params_

# in form of a dict

{'alpha': 0.1, 'l1_ratio': 1}

In [34]:
# list of other combinations 

scores = pd.DataFrame(grid_model.cv_results_)


In [43]:
scores.sort_values('rank_test_score').head(3)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
10,0.000395,0.000001,0.000196,0.000002,0.1,1,"{'alpha': 0.1, 'l1_ratio': 1}",-3.181007,-1.447729,-5.459915,-2.247365,-4.076991,-3.282601,1.401467,1
8,0.000391,0.000019,0.000191,0.000009,0.1,0.95,"{'alpha': 0.1, 'l1_ratio': 0.95}",-3.174482,-1.449040,-5.491814,-2.246541,-4.093924,-3.291160,1.413192,2
9,0.000399,0.000005,0.000196,0.000005,0.1,0.95,"{'alpha': 0.1, 'l1_ratio': 0.95}",-3.174482,-1.449040,-5.491814,-2.246541,-4.093924,-3.291160,1.413192,2


In [44]:
# if we are good with the results 

# use the following command
# it will get the best hyperparameters values internaly and run the model

grid_predict = grid_model.predict(X_test)

In [45]:
from sklearn.metrics import mean_squared_error

In [46]:
mean_squared_error(y_test, grid_predict)

2.4630652544449654